In [ ]:
import 

In [ ]:
class SpiralAnalysis(object):
    def __init__(self, list_or_path, emap):
        if isinstance(list_or_path, str):
            self.append_directory(list_or_path)
            self.append_usable_galaxies(emap)
        else:
            self.fits_list = list_or_path
            self.append_list_galaxies()
            
        self.intro_message()
    
    def append_directory(self, path):
        self.fits_list = []
        
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(".fits.gz"):
                    element = str(path) + str(file)
                    self.fits_list.append(element)
            
    
    def intro_message(self):
        print("Welcome! There are {} .FITS files in the directory.\n".format(len(self.fits_list)))
        print("There are {} usable galaxies with the percentage specified.".format(len(self.galaxy_list)))
    
    
    def append_usable_galaxies(self, emap):
        self.galaxy_list = []
        
        for path in self.fits_list:
            try:
                galaxy = SpiralGalaxy(path, emap)
            except:
                print("{} could not be initialised correctly, please check if the galaxy has known issues on the SDSS Trac website.".format(galaxy.mangaid))
                continue
            
            if galaxy.check_usability():
                self.galaxy_list.append(galaxy)
                
                
    def append_list_galaxies(self, emap):
        self.galaxy_list = []
        
        for path in self.fits_list:
            try:
                galaxy = SpiralGalaxy(path, emap)
            except:
                print("{} could not be initialised correctly, please check if the galaxy has known issues on the SDSS Trac website.".format(galaxy.mangaid))
                continue
            
            self.galaxy_list.append(galaxy)
    

    def main_df_maker(self):
        mangaid_list = []
        mass_list = []
        
        for galaxy in self.galaxy_list:            
            mangaid_list.append(galaxy.mangaid)
            mass_list.append(galaxy.get_lgmass())
        
        data = {'Galaxy': self.galaxy_list, 'MaNGA ID': mangaid_list, 'Mass': mass_list}
        self.main_df = pd.DataFrame(data)
    
    
    def forgive_me_im_a_binner(self, equal_bin = True, n = 3):
        if equal_bin:
            self.mass_bins = np.linspace(1, n, n)
            self.main_df['Mass Bins'] = pd.qcut(self.main_df['Mass'], n, labels = self.mass_bins)
        else:
            self.mass_bins = ['litte_boi', 'big_boi']
            self.main_df['Mass Bins'] = pd.cut(self.main_df['Mass'], bins = [- np.inf, 10.3, np.inf],
                                              labels = self.mass_bins)
    
    
    def append_group_data(self, galaxy_array):
        group_sprial_sfr = []
        group_spiral_radius = []
        group_spiral_variance = []

        group_nonsprial_sfr = []
        group_nonspiral_radius = []
        group_nonspiral_variance = []

        for galaxy in galaxy_array:
            galaxy.get_sfr_lists()

            group_sprial_sfr = np.append(group_sprial_sfr, galaxy.spiral_sfr)
            group_spiral_radius = np.append(group_spiral_radius, galaxy.spiral_radius)
            group_spiral_variance = np.append(group_spiral_variance, galaxy.spiral_sfr_variance)

            group_nonsprial_sfr = np.append(group_nonsprial_sfr, galaxy.nonspiral_sfr)
            group_nonspiral_radius = np.append(group_nonspiral_radius, galaxy.nonspiral_radius)
            group_nonspiral_variance = np.append(group_nonspiral_variance, galaxy.nonspiral_sfr_variance)

        return [(group_sprial_sfr, group_spiral_radius, group_spiral_variance),
                (group_nonsprial_sfr, group_nonspiral_radius, group_nonspiral_variance)]
    
    
    def group_df_maker(self, group_data):        
        spiral_DF = pd.DataFrame({'Spiral SFR': group_data[0][0],
                                  'Spiral Radius': group_data[0][1],
                                  'Spiral SFR Variance': group_data[0][2]})

        nonspiral_DF = pd.DataFrame({'Non Spiral SFR': group_data[1][0],
                                     'Non Spiral Radius': group_data[1][1],
                                     'Non Spiral SFR Variance': group_data[1][2]})

        return spiral_DF, nonspiral_DF
        
        
    def data_maker(self):
        self.main_df_maker()
        self.forgive_me_im_a_binner()
        mass_binned_galaxies = [self.main_df[self.main_df['Mass Bins'] == x] for x in self.mass_bins]
        
        group_data = []
        for group in mass_binned_galaxies:
            data = self.append_group_data(group.Galaxy)
            spiral_df, nonspiral_df = group_df_maker(data)
            
            spiral_df['Spiral Bins'] = pd.cut(spiral_df['Spiral Radius'], [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
            nonspiral_df['Non Spiral Bins'] = pd.cut(nonspiral_df['Non Spiral Radius'], [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
            
            avg_spiral_sfr = spiral_df.groupby('Spiral Bins')['Spiral SFR'].mean().values
            avg_spiral_variance = spiral_df.groupby('Spiral Bins')['Spiral SFR Variance'].mean().values
            
            avg_nonspiral_sfr = nonspiral_df.groupby('Non Spiral Bins')['Non Spiral SFR'].mean().values
            avg_nonspiral_variance = nonspiral_df.groupby('Non Spiral Bins')['Non Spiral SFR Variance'].mean().values
            
            group_data.append([avg_spiral_sfr, avg_spiral_variance, avg_nonspiral_sfr, avg_nonspiral_variance])
            
        return group_data

In [1]:
import pickle
with open("test.txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)

In [4]:
p = SpiralAnalysis(b, 'emline_ha_gflux_6564')

NameError: name 'SpiralAnalysis' is not defined